In [ ]:
import logging

logging.getLogger("rubin_nights").setLevel(logging.INFO)

# This cell is only for setting example parameter defaults - gets replaced by sidecar.
n_nights = 14
day_obs_max = "20260128"
not_times_square = True

# Visit acquisition rate and delivered image quality from {{ params.day_obs_min }} to {{ params.day_obs_max }} # 

Evaluate simply number of science visits and delivered image quality compared to average baseline v5.1 numbers.

In [ ]:
try:
    not_times_square
except NameError:
    not_times_square = False

if not not_times_square:
    print("updating rubin_nights")
    !pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git  --no-deps  > /dev/null 2>&1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

from astropy.time import Time, TimeDelta
import astropy.units as u

import warnings

from rubin_scheduler.site_models import SeeingModel

from rubin_nights import connections
from rubin_nights.augment_visits import augment_visits
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.dayobs_utils as rn_dayobs


from rubin_nights.reference_values import (
    SCIENCE_PROGRAMS,
    SIGMA_TO_FWHM,
    PLATESCALE,
)
from rubin_nights.dayobs_utils import estimated_baseline_visit_range

In [ ]:
# Turn day_obs limits into astropy times

t_start = Time(
    f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format="isot", scale="utc"
) - TimeDelta(n_nights - 1, format="jd")
t_end = Time(
    f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format="isot", scale="utc"
) + TimeDelta(1, format="jd")

one_day = TimeDelta(1, format="jd")
days = t_start + one_day * np.arange(0, (t_end - t_start).jd)
day_obs_list = [
    rn_dayobs.day_obs_str_to_int(rn_dayobs.time_to_day_obs(d)) for d in days
]

day_obs_min = day_obs_list[0]

if n_nights == 1:
    print(
        f"Evaluating from {day_obs_min} to {day_obs_max}, a span of {n_nights} night."
    )
else:
    print(
        f"Evaluating from {day_obs_min} to {day_obs_max}, a span of {n_nights} nights"
    )

# First - query consdb for visit information
endpoints = connections.get_clients()

In [ ]:
# refresh_visits and not_times_square just make it easier to rerun the notebook yourself without re-downloading info
refresh_visits = True
if refresh_visits:
    visits = endpoints["consdb_tap"].get_visits(
        "lsstcam",
        t_start,
        t_end,
        visit_constraint=f"science_program in {SCIENCE_PROGRAMS}",
        augment=True,
    )
    if not_times_square:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            visits.to_hdf("v_now.h5", key="visits")
else:
    visits = pd.read_hdf("v_now.h5")

# Let the user know if things went as expected
print(
    f"Retrieved {len(visits)} science visits from consdb between {t_start.iso} and {t_end.iso}"
)

if len(visits) == 0:
    print(
        "Since 0 visits were retrieved, the rest of this notebook will fail and give errors."
    )
    print(
        "Not retrieving visits could be a service failure or it could indicate no visits present during this timespan."
    )


# Pop this here so we have informative print statements in one place
ave_n_vis_per_night = (
    visits.groupby("day_obs").agg({"visit_id": "count"}).visit_id.mean()
)
print(f"Average number of visits per night: {ave_n_vis_per_night:.2f}")

In [ ]:
# Add estimate of expected visits, from ensemble of v5.1 simulations and a fit
dd = []
for day_obs in day_obs_list:
    dd.append(estimated_baseline_visit_range(day_obs))
nvis_expected = pd.DataFrame(dd, index=day_obs_list)
print(
    f"Total number of estimated expected visits would be {nvis_expected.n_vis_ave.sum()} between {t_start.iso} and {t_end.iso}"
)
print(f"Average number estimated per night {nvis_expected.n_vis_ave.mean():.2f}")

In [ ]:
# Join acquired number of visits per day with expected.
acq_per_day = (
    visits.groupby("day_obs")
    .agg({"visit_id": "count"})
    .rename({"visit_id": "n_vis_acq"}, axis=1)
)
nvis_per_day = nvis_expected.join(acq_per_day, how="outer").fillna(0)
nvis_per_day["ratio"] = nvis_per_day["n_vis_acq"] / nvis_per_day["n_vis_ave"]
mean_nvis_acq_expected = nvis_per_day.ratio.mean()
print(
    f"Comparing acquired science visits per night to the estimated expected number of visits per night - mean ratio is {mean_nvis_acq_expected:.2f}"
)
print(
    f"Comparing the total bulk number of visits (not per night) we've acquired {len(visits) / nvis_expected.n_vis_ave.sum():.2f} of what we might have."
)

# And here, we have to assume that the fraction of the night we tried to run the FBS was the full night, since we don't know otherwise.

In [ ]:
# What would the fE box approximately translate to?

print("Using baseline_v5.1.0_10yrs as the comparison")
fraction_wfd = 1692518.00 / 2075536.00
fO_comparison = 762.00 / 825 * 0.9 / fraction_wfd
print(f"fO comparison - median_nvis / 825 * 0.9 / fraction_wfd = {fO_comparison:.2f}")

# nvis in sim vs. nvis calculated from estimate above if on-sky all the time
v51_nvisits = 2075536
total_est_nvisits = 3269149  # for 365 nights a year * 10 years
# total_est_nvisits *= (365 - 5*7) / 365 # cut out some fraction of nights for scheduled downtime to help cover bunchiness
ratio_sim_nvis = v51_nvisits / total_est_nvisits
print(
    f"In v5.1 we have ~{v51_nvisits/1e6:.1f}M visits; sum of the calculated visits per night over 10 years is ~{total_est_nvisits/1e6:.1f}M visits; ratio {ratio_sim_nvis:.2f}"
)

# Assuming fA = 0.99
fA = 0.99
print(f"Assuming fA = {fA:.2f}")


# And fill in a speculative value for fS for now
def fS(atm, sys):
    fwhm = np.sqrt(atm**2 + sys**2)
    delta_m = 2.5 * np.log10(0.7 / fwhm)
    f_s = 10 ** (0.8 * delta_m)
    return f_s


fS = 1.3 * (fS(0.6, 0.5) / fS(0.6, 0.4))
fS = 1.0
print(f"And approximating fS as {fS:.2f}")

fE = fO_comparison * fA * fS
print(f"Means these combine to fE {fE:.2f}")

fE_nvis_ratio = ratio_sim_nvis / fO_comparison / fS / fA
print(f"Translating this to n_vis_acquired / n_vis_expected: {fE_nvis_ratio:.2f}")

fE_min = 0.7

min_fE_nvis_ratio = ratio_sim_nvis / fO_comparison / fS / fA * fE_min
print(
    f"With a minimum ratio of n_vis_acquired / n_vis_expected of {min_fE_nvis_ratio:.2f}"
)

In [ ]:
# fig, ax = plt.subplots(1,1,figsize=(10, 6))
# day_obs = 20260128
# q = visits.query("day_obs == @day_obs")
# tstart = Time(q.obs_start_mjd.min() - 30/60/24, format='mjd')
# tend = Time(q.obs_end_mjd.max() + 30/60/24, format='mjd')
# dimm = endpoints['efd'].select_time_series("lsst.sal.DIMM.logevent_dimmMeasurement", ['fwhm'], tstart, tend, index=1)
# if len(dimm) > 0:
#     ax.plot(dimm.index, dimm.fwhm, linestyle='-', marker='.', linewidth=1, label="dimm1")
#     print(f"Dayobs {day_obs}")
#     print(f"DIMM mean FWHM {dimm.fwhm.mean():.2f}")
# dimm2 = endpoints['efd'].select_time_series("lsst.sal.DIMM.logevent_dimmMeasurement", ['fwhm'], tstart, tend, index=2)
# if len(dimm2) > 0:
#     ax.plot(dimm2.index, dimm2.fwhm, linestyle='-', marker='.',  linewidth=1, label="dimm2", alpha=0.5)
# ringss = endpoints['efd'].select_time_series("lsst.sal.ESS.logevent_ringssMeasurement", ['fwhmFree', 'fwhmSector', 'fwhmScintillation'], tstart, tend)
# if len(ringss) > 0:
#     ax.plot(ringss.index, ringss.fwhmFree, alpha=0.5, linestyle='-', marker='.',  label="ringss")

# ax.plot(Time(q.exp_midpt_mjd, format='mjd').datetime, q.atm_500_zenith,
#         marker='o', markeredgecolor='r', markerfacecolor='none', linestyle='',  label="calculated Atm500Zenith")
# #ax.plot(Time(q.exp_midpt_mjd, format='mjd').datetime, q.airmass, marker='*', markeredgecolor='k', markerfacecolor='none', linestyle='', label="Airmass")
# #ax.plot(Time(q.exp_midpt_mjd, format='mjd').datetime, q.fwhm_eff, marker='*', markerfacecolor='none', linestyle='', label="FWHM")
# ax.set_xlabel("MJD")
# ax.set_ylabel("Arcseconds")
# _ = ax.legend(loc=(1.01, 0.2))
# ax.grid(alpha=0.3)

In [ ]:
# what would system contribution to image quality box translate to?
dome_fwhm = 0.084
idiq = 0.39
min_idiq = np.sqrt(0.45**2 - dome_fwhm**2)
print(
    f"Goal instrumental contribution at zenith {idiq:.2f} (min acceptable {min_idiq:.2f})"
)

print(f"Dome contribution {dome_fwhm:.3f}")
fiducial_atm = 0.7
min_diq = np.sqrt(fiducial_atm**2 + min_idiq**2 + dome_fwhm**2)
diq = np.sqrt(fiducial_atm**2 + idiq**2 + dome_fwhm**2)
print(
    f"Translate fiducial atmospheric contribution of {fiducial_atm:0.2f} arcsec to DIQ of {diq:.2f} (500nm, zenith) (min: {min_diq:.2f})"
)
diq_x = np.sqrt((fiducial_atm * np.power(1.2, 0.6)) ** 2 + idiq**2)
min_diq_x = np.sqrt((fiducial_atm * np.power(1.2, 0.6)) ** 2 + min_idiq**2)
print(f"Same diq at X=1.2: {diq_x:.2f} (min: {min_diq_x:.2f})")

print(
    f"Comparison point: calculated atm_500_zenith median is {visits.atm_500_zenith.median():.2f}"
)
print(
    f"and median of psf fwhm values corrected to 500nm zenith is {visits.fwhm_500_zenith.median():.2f}"
)
print(
    f"Median DIMM1 value (when available): {visits.query('dimm_seeing<10').dimm_seeing.median():.2f}"
)
print(
    f"Median DIQ is {visits.fwhm_eff.median():.2f}; Mean {visits.fwhm_eff.mean():.2f}"
)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
q = visits.query("band != 'y'")
ax = axes[0]
ax.hexbin(q.idiq_aos_cam, q.idiq_donut_blur, mincnt=1, gridsize=200, vmin=1, vmax=10)
x = np.arange(q.idiq_aos_cam.min(), q.idiq_aos_cam.max(), 0.1)
ax.plot(x, x, "r:")
ax.grid(alpha=0.3)
ax.set_xlabel("iDIQ from aos_cam")
ax.set_ylabel("iDIQ from donut_blur")
ax.set_title(f"science visits {visits.day_obs.min()} {visits.day_obs.max()}")

ax = axes[1]
q = visits.query("dimm_seeing < 10")  # and band != 'y'")
ax.hexbin(q.dimm_seeing, q.atm_500_zenith, mincnt=1, gridsize=150, vmin=1, vmax=5)
x = np.arange(q.atm_500_zenith.min(), q.atm_500_zenith.max(), 0.1)
ax.plot(x, x, "r:")
ax.grid(alpha=0.3)
ax.set_xlabel("DIMM1")
ax.set_ylabel("Atm 500 zenith (calculated)")
_ = ax.set_title(f"science visits {visits.day_obs.min()} {visits.day_obs.max()}")

In [ ]:
fwhm_500_zenith_median = visits.fwhm_500_zenith.median()
fwhm_median = visits.fwhm_geom.median()
idiq_aos_cam_median = visits.idiq_aos_cam.median()

jitter = np.random.uniform(low=-0.03, high=0.03, size=len(nvis_per_day))

g = visits.groupby("day_obs")

upper = 0.75
lower = 0.25
daily_fwhm = pd.DataFrame(
    {
        "fwhm_500_zenith": g.fwhm_500_zenith.median(),
        "fwhm_500_zenith_up_rr": g.fwhm_500_zenith.quantile(upper)
        - g.fwhm_500_zenith.quantile(0.5),
        "fwhm_500_zenith_lo_rr": g.fwhm_500_zenith.quantile(0.5)
        - g.fwhm_500_zenith.quantile(lower),
        "fwhm_geom": g.fwhm_geom.median(),
        "fwhm_geom_up_rr": g.fwhm_geom.quantile(upper) - g.fwhm_geom.quantile(0.5),
        "fwhm_geom_lo_rr": g.fwhm_geom.quantile(0.5) - g.fwhm_geom.quantile(lower),
        "idiq_aos_cam": g.idiq_aos_cam.median(),
        "idiq_aos_cam_up_rr": g.idiq_aos_cam.quantile(upper)
        - g.idiq_aos_cam.quantile(0.5),
        "idiq_aos_cam_lo_rr": g.idiq_aos_cam.quantile(0.5)
        - g.idiq_aos_cam.quantile(lower),
    }
)

daily = nvis_per_day.join(daily_fwhm, how="outer")

# Create FWHM values for plot
daily["fwhm_500_zenith_filled"] = np.where(
    daily.fwhm_500_zenith.fillna(-9).values < 0,
    fwhm_500_zenith_median + jitter,
    daily.fwhm_500_zenith.values,
)
daily["fwhm_geom_filled"] = np.where(
    daily.fwhm_geom.fillna(-9).values < 0, fwhm_median + jitter, daily.fwhm_geom.values
)
daily["idiq_aos_cam_filled"] = np.where(
    daily.idiq_aos_cam.fillna(-9).values < 0,
    idiq_aos_cam_median + jitter,
    daily.idiq_aos_cam.values,
)

for c in daily.columns:
    if c.endswith("_rr"):
        daily[c] = daily[c].fillna(0)

daily["alpha"] = np.linspace(0.1, 1.0, len(daily))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8))

# Plot the daily points
ax.scatter(
    daily.ratio,
    daily.idiq_aos_cam_filled,
    s=75,
    color="gray",
    facecolor="black",
    marker="o",
    alpha=daily.alpha.values,
)
dl = daily.iloc[-1]
ax.scatter(
    dl.ratio,
    dl.idiq_aos_cam_filled,
    s=75,
    color="gray",
    facecolor="black",
    marker="o",
    alpha=dl.alpha,
    label="Daily Values",
)
ax.errorbar(
    daily.ratio,
    daily.idiq_aos_cam_filled,
    yerr=[daily.idiq_aos_cam_lo_rr, daily.idiq_aos_cam_up_rr],
    color="lightgray",
    linewidth=3,
    linestyle="",
    marker="",
    zorder=0,
)

ax.plot(
    mean_nvis_acq_expected,
    idiq_aos_cam_median,
    "o",
    color="black",
    markerfacecolor="none",
    markersize=22,
    label="Over Time Period",
)
ax.plot(
    fE_nvis_ratio,
    idiq,
    "*",
    color="gray",
    markerfacecolor="orange",
    markersize=25,
    label="Goal",
)
plt.legend(fontsize="x-large", labelspacing=1.2)

x = np.arange(min_fE_nvis_ratio, 1.2, 0.01)
y1 = 0.1
y2 = min_idiq
ax.fill_between(x, y1=y1, y2=y2, color="green", alpha=0.3)

ax.set_xlim(0, 1.1)
ax.set_xlabel("Number of visits / expected number (per night)", fontsize="xx-large")
ax.set_ylim(0.3, 0.7)
ax.set_ylabel("Median iDIQ", fontsize="xx-large")
ax.tick_params("both", labelsize="large", length=6)
ax.grid(alpha=0.3)

_ = ax.set_title(f"{day_obs_min} to {day_obs_max}", fontsize="xx-large")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8))

# Plot the daily points
ax.scatter(
    daily.ratio,
    daily.fwhm_geom_filled,
    s=75,
    color="gray",
    facecolor="black",
    marker="o",
    alpha=daily.alpha.values,
)
dl = daily.iloc[-1]
ax.scatter(
    dl.ratio,
    dl.fwhm_geom_filled,
    s=75,
    color="gray",
    facecolor="black",
    marker="o",
    alpha=dl.alpha,
    label="Daily Values",
)
ax.errorbar(
    daily.ratio,
    daily.fwhm_geom_filled,
    yerr=[daily.fwhm_geom_lo_rr, daily.fwhm_geom_up_rr],
    color="lightgray",
    linewidth=3,
    linestyle="",
    marker="",
    zorder=0,
)

ax.plot(
    mean_nvis_acq_expected,
    fwhm_median,
    "o",
    color="black",
    markerfacecolor="none",
    markersize=22,
    label="Over Time Period",
)
ax.plot(
    fE_nvis_ratio,
    diq_x,
    "*",
    color="gray",
    markerfacecolor="orange",
    markersize=25,
    label="Goal",
)
plt.legend(fontsize="x-large", labelspacing=1.2)

x = np.arange(min_fE_nvis_ratio, 1.2, 0.01)
y1 = 0.1
y2 = min_diq_x
ax.fill_between(x, y1=y1, y2=y2, color="green", alpha=0.3)

ax.set_xlim(0, 1.1)
ax.set_xlabel("Number of visits / expected number (per night)", fontsize="xx-large")
ax.set_ylim(0.6, None)
ax.set_ylabel("Median fwhm", fontsize="xx-large")
ax.tick_params("both", labelsize="large", length=6)
ax.grid(alpha=0.3)

_ = ax.set_title(f"{day_obs_min} to {day_obs_max}", fontsize="xx-large")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8))

# Plot the daily points
ax.scatter(
    daily.ratio,
    daily.fwhm_500_zenith_filled,
    s=75,
    color="gray",
    facecolor="black",
    marker="o",
    alpha=daily.alpha.values,
)
dl = daily.iloc[-1]
ax.scatter(
    dl.ratio,
    dl.fwhm_500_zenith_filled,
    s=75,
    color="gray",
    facecolor="black",
    marker="o",
    alpha=dl.alpha,
    label="Daily Values",
)
ax.errorbar(
    daily.ratio,
    daily.fwhm_500_zenith_filled,
    yerr=[daily.fwhm_500_zenith_lo_rr, daily.fwhm_500_zenith_up_rr],
    color="lightgray",
    linewidth=3,
    linestyle="",
    marker="",
    zorder=0,
)

ax.plot(
    mean_nvis_acq_expected,
    fwhm_500_zenith_median,
    "o",
    color="black",
    markerfacecolor="none",
    markersize=22,
    label="Over Time Period",
)
ax.plot(
    fE_nvis_ratio,
    diq,
    "*",
    color="gray",
    markerfacecolor="orange",
    markersize=25,
    label="Goal",
)
plt.legend(fontsize="x-large", labelspacing=1.2)

x = np.arange(min_fE_nvis_ratio, 1.2, 0.01)
y1 = 0.1
y2 = min_diq
ax.fill_between(x, y1=y1, y2=y2, color="green", alpha=0.3)

ax.set_xlim(0, 1.1)
ax.set_xlabel("Number of visits / expected number (per night)", fontsize="xx-large")
ax.set_ylim(0.6, None)
ax.set_ylabel(
    "Median fwhm corrected to 500nm @ zenith (arcseconds)", fontsize="xx-large"
)
ax.tick_params("both", labelsize="large", length=6)
ax.grid(alpha=0.3)

_ = ax.set_title(f"{day_obs_min} to {day_obs_max}", fontsize="xx-large")

In [ ]:
plt.figure(figsize=(15, 6))
x = np.arange(0, len(daily))
plt.plot(x, daily.ratio, marker=".", label="nvisits acquired/expected")
plt.plot(x, daily.idiq_aos_cam, marker=".", label="median iDIQ")
plt.legend()
plt.ylim(0, 1)
plt.grid(alpha=0.3)
_ = plt.xticks(x, labels=daily.index, rotation=90)